In [13]:
# 필요한 라이브러리 설치 (필요시)
# !pip install FinanceDataReader ta-lib pandas-ta plotly

import FinanceDataReader as fdr
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import talib
from datetime import datetime, timedelta

In [14]:
def calculate_ichimoku(df):
    """일목균형표 계산"""
    # 전환선 (Conversion Line): 9일 기준
    high_9 = df['High'].rolling(window=9).max()
    low_9 = df['Low'].rolling(window=9).min()
    df['Tenkan'] = (high_9 + low_9) / 2
    
    # 기준선 (Base Line): 26일 기준
    high_26 = df['High'].rolling(window=26).max()
    low_26 = df['Low'].rolling(window=26).min()
    df['Kijun'] = (high_26 + low_26) / 2
    
    # 선행스팬 A (Leading Span A): (전환선 + 기준선) / 2, 26일 선행
    df['Senkou_A'] = ((df['Tenkan'] + df['Kijun']) / 2).shift(26)
    
    # 선행스팬 B (Leading Span B): 52일 기준, 26일 선행
    high_52 = df['High'].rolling(window=52).max()
    low_52 = df['Low'].rolling(window=52).min()
    df['Senkou_B'] = ((high_52 + low_52) / 2).shift(26)
    
    # 후행스팬 (Lagging Span): 종가를 26일 후행
    df['Chikou'] = df['Close'].shift(-26)
    
    return df

In [15]:
def detect_patterns(df):
    """TA-Lib을 이용한 캔들 패턴 인식"""
    patterns = {}
    
    # 주요 상승 패턴들
    bullish_patterns = {
        'Hammer': talib.CDLHAMMER,
        'Inverted Hammer': talib.CDLINVERTEDHAMMER,
        'Morning Star': talib.CDLMORNINGSTAR,
        'Three White Soldiers': talib.CDL3WHITESOLDIERS,
        'Bullish Engulfing': talib.CDLENGULFING,
        'Piercing': talib.CDLPIERCING,
        'Bullish Harami': talib.CDLHARAMI,
        'Dragonfly Doji': talib.CDLDRAGONFLYDOJI,
    }
    
    # 하락 패턴들
    bearish_patterns = {
        'Hanging Man': talib.CDLHANGINGMAN,
        'Shooting Star': talib.CDLSHOOTINGSTAR,
        'Evening Star': talib.CDLEVENINGSTAR,
        'Three Black Crows': talib.CDL3BLACKCROWS,
        'Bearish Engulfing': talib.CDLENGULFING,
        'Dark Cloud Cover': talib.CDLDARKCLOUDCOVER,
        'Bearish Harami': talib.CDLHARAMI,
    }
    
    # 패턴 감지
    for name, func in bullish_patterns.items():
        result = func(df['Open'], df['High'], df['Low'], df['Close'])
        if result.any():
            patterns[name] = result
    
    for name, func in bearish_patterns.items():
        result = func(df['Open'], df['High'], df['Low'], df['Close'])
        if result.any():
            patterns[name] = result
    
    return patterns

In [16]:
def create_stock_chart(ticker, stock_name, period_days=180):
    """
    주식 차트 생성 함수
    
    Parameters:
    - ticker: 주식 티커 (예: '005930' - 삼성전자)
    - stock_name: 종목명
    - period_days: 조회 기간 (기본 180일)
    """
    
    # 데이터 가져오기
    end_date = datetime.now()
    start_date = end_date - timedelta(days=period_days + 100)  # 이동평균선 계산을 위해 여유있게
    
    print(f"📊 {stock_name} ({ticker}) 데이터 불러오는 중...")
    df = fdr.DataReader(ticker, start_date, end_date)
    
    if df is None or len(df) == 0:
        print("❌ 데이터를 가져올 수 없습니다.")
        return
    
    # 이동평균선 계산
    df['MA5'] = df['Close'].rolling(window=5).mean()
    df['MA20'] = df['Close'].rolling(window=20).mean()
    df['MA60'] = df['Close'].rolling(window=60).mean()
    df['MA112'] = df['Close'].rolling(window=112).mean()
    df['MA224'] = df['Close'].rolling(window=224).mean()
    
    # 일목균형표 계산
    df = calculate_ichimoku(df)
    
    # 패턴 감지
    patterns = detect_patterns(df)
    
    # 최근 period_days만 표시
    df_display = df.tail(period_days)
    
    # 차트 생성
    fig = go.Figure()
    
    # 캔들스틱 차트 (상승: 빨강, 하락: 파랑)
    fig.add_trace(go.Candlestick(
        x=df_display.index,
        open=df_display['Open'],
        high=df_display['High'],
        low=df_display['Low'],
        close=df_display['Close'],
        name='Price',
        increasing_line_color='red',
        decreasing_line_color='blue',
        increasing_fillcolor='rgba(255, 0, 0, 0.5)',
        decreasing_fillcolor='rgba(0, 0, 255, 0.5)'
    ))
    
    # 이동평균선 추가
    colors = {
        'MA5': 'purple',
        'MA20': 'orange', 
        'MA60': 'green',
        'MA112': 'brown',
        'MA224': 'pink'
    }
    
    for ma_name, color in colors.items():
        fig.add_trace(go.Scatter(
            x=df_display.index,
            y=df_display[ma_name],
            name=ma_name,
            line=dict(color=color, width=1.5)
        ))
    
    # 일목균형표 추가
    # 전환선 (빨강)
    fig.add_trace(go.Scatter(
        x=df_display.index,
        y=df_display['Tenkan'],
        name='전환선',
        line=dict(color='red', width=1, dash='dot')
    ))
    
    # 기준선 (파랑)
    fig.add_trace(go.Scatter(
        x=df_display.index,
        y=df_display['Kijun'],
        name='기준선',
        line=dict(color='blue', width=1, dash='dot')
    ))
    
    # 선행스팬 A & B (구름)
    fig.add_trace(go.Scatter(
        x=df_display.index,
        y=df_display['Senkou_A'],
        name='선행스팬A',
        line=dict(color='rgba(0, 255, 0, 0.3)', width=1),
        fill=None
    ))
    
    fig.add_trace(go.Scatter(
        x=df_display.index,
        y=df_display['Senkou_B'],
        name='선행스팬B',
        line=dict(color='rgba(255, 0, 0, 0.3)', width=1),
        fill='tonexty',
        fillcolor='rgba(128, 128, 128, 0.2)'
    ))
    
    # 후행스팬
    fig.add_trace(go.Scatter(
        x=df_display.index,
        y=df_display['Chikou'],
        name='후행스팬',
        line=dict(color='cyan', width=1, dash='dash')
    ))
    
    # 레이아웃 설정
    fig.update_layout(
        title=f'{stock_name} ({ticker}) - 기술적 분석',
        yaxis_title='가격 (원)',
        xaxis_title='날짜',
        template='plotly_dark',
        height=800,
        xaxis_rangeslider_visible=False,
        hovermode='x unified',
        legend=dict(
            orientation="v",
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        )
    )
    
    # 패턴 정보 출력
    print(f"\n✅ 차트 생성 완료!")
    print(f"\n📈 감지된 패턴:")
    for pattern_name, pattern_data in patterns.items():
        pattern_dates = df_display.index[pattern_data.tail(period_days) != 0]
        if len(pattern_dates) > 0:
            print(f"  - {pattern_name}: {len(pattern_dates)}회 ({pattern_dates[-1].strftime('%Y-%m-%d')} 최근)")
    
    fig.show()
    
    return df, patterns

In [17]:
# 사용 예제

# 삼성전자 차트 생성 (180일)
df, patterns = create_stock_chart('005930', '삼성전자', period_days=180)

# 다른 종목 예제 (선택적으로 실행)
# df, patterns = create_stock_chart('000660', 'SK하이닉스', period_days=180)
# df, patterns = create_stock_chart('035420', 'NAVER', period_days=180)
# df, patterns = create_stock_chart('005380', '현대차', period_days=180)

📊 삼성전자 (005930) 데이터 불러오는 중...

✅ 차트 생성 완료!

📈 감지된 패턴:
  - Hammer: 3회 (2025-08-14 최근)
  - Inverted Hammer: 7회 (2025-11-13 최근)
  - Bullish Engulfing: 12회 (2025-11-04 최근)
  - Piercing: 2회 (2025-08-27 최근)
  - Bullish Harami: 22회 (2025-09-29 최근)
  - Dragonfly Doji: 1회 (2025-10-20 최근)
  - Hanging Man: 6회 (2025-10-10 최근)
  - Bearish Engulfing: 12회 (2025-11-04 최근)
  - Dark Cloud Cover: 3회 (2025-10-14 최근)
  - Bearish Harami: 22회 (2025-09-29 최근)


In [18]:
# 패턴 상세 분석 함수
def analyze_patterns_detail(df, patterns, days=30):
    """
    최근 N일 동안의 패턴을 상세 분석
    """
    print(f"\n{'='*60}")
    print(f"📊 최근 {days}일 패턴 상세 분석")
    print(f"{'='*60}\n")
    
    df_recent = df.tail(days)
    
    for pattern_name, pattern_data in patterns.items():
        pattern_recent = pattern_data.tail(days)
        detected_indices = pattern_recent[pattern_recent != 0].index
        
        if len(detected_indices) > 0:
            print(f"\n🔍 {pattern_name}:")
            for idx in detected_indices:
                date_str = idx.strftime('%Y-%m-%d')
                price = df.loc[idx, 'Close']
                signal = "🔴 상승" if pattern_recent[idx] > 0 else "🔵 하락"
                print(f"   {date_str}: {signal} (종가: {price:,.0f}원)")

# 사용 예제
# analyze_patterns_detail(df, patterns, days=30)

In [19]:
# 종목 검색 도우미 함수
def search_stock(keyword):
    """
    종목명으로 티커 검색
    """
    print(f"🔎 '{keyword}' 검색 중...\n")
    
    # KOSPI, KOSDAQ 종목 리스트 가져오기
    kospi = fdr.StockListing('KOSPI')
    kosdaq = fdr.StockListing('KOSDAQ')
    all_stocks = pd.concat([kospi, kosdaq])
    
    # 검색
    result = all_stocks[all_stocks['Name'].str.contains(keyword, na=False)]
    
    if len(result) == 0:
        print("❌ 검색 결과가 없습니다.")
        return None
    
    print(f"✅ {len(result)}개 종목 발견:\n")
    for idx, row in result.iterrows():
        print(f"  {row['Code']}: {row['Name']} ({row['Market']})")
    
    return result

# 사용 예제
# search_stock('삼성')
# search_stock('현대차')
# search_stock('NAVER')

In [20]:
"""
📖 사용 가이드
===============

1. 종목 검색:
   search_stock('삼성')

2. 차트 생성:
   df, patterns = create_stock_chart('005930', '삼성전자', period_days=180)

3. 패턴 상세 분석:
   analyze_patterns_detail(df, patterns, days=30)

📊 주요 기능:
- 캔들스틱 차트: 상승(빨강), 하락(파랑)
- 이동평균선: 5일, 20일, 60일, 112일, 224일
- 일목균형표: 전환선, 기준선, 선행스팬A/B, 후행스팬
- TA-Lib 패턴 인식: 해머, 샛별형, 3병사 등

🎯 상승 패턴:
- Hammer (해머)
- Morning Star (샛별형)
- Three White Soldiers (적삼병)
- Bullish Engulfing (상승 포괄형)
- Piercing (관통형)

📉 하락 패턴:
- Hanging Man (교수형)
- Evening Star (저녁별)
- Three Black Crows (흑삼병)
- Bearish Engulfing (하락 포괄형)
- Dark Cloud Cover (먹구름형)

💡 주요 종목 티커:
- 삼성전자: 005930
- SK하이닉스: 000660
- NAVER: 035420
- 카카오: 035720
- 현대차: 005380
- 삼성바이오로직스: 207940
- LG에너지솔루션: 373220
"""

"\n📖 사용 가이드\n===============\n\n1. 종목 검색:\n   search_stock('삼성')\n\n2. 차트 생성:\n   df, patterns = create_stock_chart('005930', '삼성전자', period_days=180)\n\n3. 패턴 상세 분석:\n   analyze_patterns_detail(df, patterns, days=30)\n\n📊 주요 기능:\n- 캔들스틱 차트: 상승(빨강), 하락(파랑)\n- 이동평균선: 5일, 20일, 60일, 112일, 224일\n- 일목균형표: 전환선, 기준선, 선행스팬A/B, 후행스팬\n- TA-Lib 패턴 인식: 해머, 샛별형, 3병사 등\n\n🎯 상승 패턴:\n- Hammer (해머)\n- Morning Star (샛별형)\n- Three White Soldiers (적삼병)\n- Bullish Engulfing (상승 포괄형)\n- Piercing (관통형)\n\n📉 하락 패턴:\n- Hanging Man (교수형)\n- Evening Star (저녁별)\n- Three Black Crows (흑삼병)\n- Bearish Engulfing (하락 포괄형)\n- Dark Cloud Cover (먹구름형)\n\n💡 주요 종목 티커:\n- 삼성전자: 005930\n- SK하이닉스: 000660\n- NAVER: 035420\n- 카카오: 035720\n- 현대차: 005380\n- 삼성바이오로직스: 207940\n- LG에너지솔루션: 373220\n"